In [1]:
"""Use of shanghai_platform
one simulation, several draws"""




#TODO

#put in the module
#create new file for automation

'Use of shanghai_platform\none simulation, several draws'

In [2]:
# 0.1 Modules

from CircleNet.shape import circle,Timer
from CircleNet.shanghai_platform import Simulation, MatchingPlatform, Driver, Passenger
from  CircleNet.shanghai_platform_extension import Positions_drawing, plot_routes, plot_relative_routes

In [3]:
# 0.2 Help

#help(MatchingPlatform)
#print(Driver.attributes), print(Passenger.attributes)

In [4]:
# 1.1 Parameters

speed=25 #km/h
R=25 #km
end=3 #h

N_driver=600
N_passenger=600


#in second :
#Drivers:
watching_repetition_time=60
first_watching_before_first_departure=5*60
window_size_of_departure=15*60
time_elasticity=5*60
fuel_cost=0.5
time_perception=5

#Passenger
publishing_advance=20*60

In [5]:
# 1.2 Simulation type

#transformation in m, s
speed/=3.6
R*=1000
end*=3600

def incentive(b):
    def o(*a,**k):
        return b(*a,**k)+500
    return o

#benefit for a drive
@incentive
def benefits(origin,destination,network):
    """Shanghai pricing"""
    distance=network.travel_distance(origin,destination)
    if distance < 3000: #3n first km fixprice
        return 11#in RMB
    if distance < 20000:#until 20km at 1.5RMB / km
        return 6.5 + distance * 1.5 # 11 + (distance - 3) * 1.5
    return 8.5 + distance # 28.5 + (distance - 20) *1   # 28.5 = 3 + (20 - 3) * 1.5



#agents generator
def SimpleDriver(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    w=(t+first_watching_before_first_departure,t+first_watching_before_first_departure+window_size_of_departure)
    A=w[1]+simulation.network.travel_time(O,D)+time_elasticity
    return Driver(first_watching_time=t,
                  repetition_time=watching_repetition_time,
                  departure_window=w,
                  position=O,destination=D,
                  last_arrival_time=A,
                  fuel_cost=fuel_cost,
                  time_perception=time_perception)
def SimplePassenger(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Passenger(publishing_time=t,
                     last_departure_time=t+publishing_advance,
                     position=O,destination=D)


In [6]:
#1.3 Simulation construction

N=circle(R,speed)
T=Timer(end)
simu=Simulation(N,T)


simu.matchingAlgo=MatchingPlatform(benefits,simu)


for i in range(N_driver):
    simu.add(SimpleDriver(simu))
for i in range(N_passenger):
    simu.add(SimplePassenger(simu))
    



In [7]:
#2. Execution

#simu.set_action(print,)
simu()

In [52]:
#3.1 Observations

from CircleNet.shanghai_platform_extension import driver_matched_route_extraction

#percentage of efficiency
nb_driver_alone=0
for p in simu:
    if isinstance(p,Driver):
        for t,action in p.story:
            if action[0] is "alone":
                nb_driver_alone+=1
                break
print(nb_driver_alone,"/",N_driver)

nb_passenger_missed=0
for p in simu:
    if isinstance(p,Passenger):
        for action in p.story.actions:
            if action[0] is "missed":
                nb_passenger_missed+=1
                break
print(nb_passenger_missed,"/",N_passenger)

#efficiency
def nb_match(simu):
    out=0
    for p in simu:
        if isinstance(p,Driver):
            for _,action in p.story:
                if action[0] is "matched":
                    out+=1
                    break 
    return out

#vks
def vks(trajectory,network):
    out=network.travel_distance(trajectory[0],trajectory[3])
    out-=network.travel_distance(trajectory[0],trajectory[1]) + network.travel_distance(trajectory[2],trajectory[3])
    return out
def vks_average_total(simu):
    l=driver_matched_route_extraction(simu)
    l=[vks(t,simu.network) for t in l]
    return sum(l)/len(l),sum(l)

#wait
def waiting_times(simu):
    matched =[]
    for p in simu:
        if isinstance(p,Passenger):
            for t,action in p.story:
                if action[0] is "matched":
                    for t2,action2 in p.story:
                        if action2[0] is "waiting":
                            matched.append(t-t2)
                            break
                    break
    return sum(matched)/len(matched)

#summary
def extract_result_data(simu,data):#data already gather parameters
    """has results extracted from the simulation to the dictionary of parameters
    need nb_driver, nb_passenger"""
    assert "nb_driver" in data
    assert "nb_passenger" in data
    data["average_waiting_time"]=waiting_times(simu)
    data["average_vks"],results["total_vks"]=vks_average_total(simu)
    data["nb_match"]=nb_match(simu)
    data["driver_efficiency"]=data["nb_match"]/data["nb_driver"]
    data["passenger_efficiency"]=data["nb_match"]/data["nb_passenger"]
    return data
    

results={}
results["nb_driver"]=N_driver
results["nb_passenger"]=N_passenger
results=extract_result_data(simu,results)
print(results)

422 / 600
422 / 600
{'passenger_efficiency': 0.2966666666666667, 'average_waiting_time': 90.63929389411021, 'driver_efficiency': 0.2966666666666667, 'nb_passenger': 600, 'nb_driver': 600, 'total_vks': 2435462.9759513247, 'average_vks': 13682.376269389464, 'nb_match': 178}


In [9]:
# 3.2 Animation

dr=Positions_drawing(simu)
dr.set_options(margin=20,real_size=(10,10),time_interval=30,time_coef=1000,repetition=True,hide_axes=True)
#%matplotlib notebook
dr()

In [10]:
# 3.3 Route representation


#simple route plot

plot_routes(simu)





In [11]:

#aggregated

plot_relative_routes(simu)



